In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
#importing libraries

# Import library modules
import sys
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt
import scipy
from scipy import optimize 
from pylab import *
from PIL import Image  # Python Imaging Library
# The following is a substitute for cv2.imshow, 
#  which you would use on your local machine but Colab does not support it
# from google.colab.patches import cv2_imshow

print('Python version:', sys.version)
print('OpenCV version:', cv2.__version__)
print('NumPy version: ', np.__version__)

Python version: 3.7.13 (default, Mar 28 2022, 08:03:21) [MSC v.1916 64 bit (AMD64)]
OpenCV version: 4.6.0
NumPy version:  1.21.5


In [2]:
face_cascade = cv2.CascadeClassifier(r"C:\Users\Amri\Downloads\AML_Project_model and hc\HaarCascade\haarcascade_frontalface_default.xml")
eye_cascade1 = cv2.CascadeClassifier(r"C:\Users\Amri\Downloads\AML_Project_model and hc\HaarCascade\haarcascade_eye.xml")
eye_cascade2 = cv2.CascadeClassifier(r"C:\Users\Amri\Downloads\AML_Project_model and hc\HaarCascade\haarcascade_eye_tree_eyeglasses.xml")
model = load_model(r"C:\Users\Amri\Downloads\AML_Project_model and hc\BestModel")

In [ ]:
cap = cv2.VideoCapture(0)
# Score = 0
while True:
    ret, frame = cap.read()
    height = frame.shape[0]
    width = frame.shape[1]
    equ = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hist,bins = np.histogram(frame.flatten(),256,[0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * float(hist.max()) / cdf.max()
    gray = cv2.equalizeHist(equ)

    faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
    eyes= eye_cascade2.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)
    
    cv2.rectangle(frame, (0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color= (255,0,0), thickness=3 )
        
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color= (255,0,0), thickness=3 )
        
        # preprocessing steps
        eye= frame[ey:ey+eh,ex:ex+ew]
        eye= cv2.resize(eye,(80,80))
        eye= eye/255
        eye= eye.reshape(80,80,3)
        eye= np.expand_dims(eye,axis=0)
        # preprocessing is done now model prediction
        prediction = model.predict(eye)
        
        # if eyes are closed
        if prediction[0][0]>0.30:
            cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            # cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                      #  thickness=1,lineType=cv2.LINE_AA)
            # Score=Score+1
            # if(Score>15):
            #     try:
            #         sound.play()
            #     except:
            #         pass
            
        # if eyes are open
        elif prediction[0][1]>0.90:
            cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)      
            # cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
            #            thickness=1,lineType=cv2.LINE_AA)
            # Score = Score-1
            # if (Score<0):
            #     Score=0
            
        
    cv2.imshow("frame",frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 32ms/step
